In [ ]:
import os
import json
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import utils
from models.trajfm import TrajFM
from data import TrajFMDataset, PretrainPadder, fetch_task_padder, X_COL, Y_COL, coord_transform_GPS_UTM
from torch.utils.data import random_split
import warnings
from pipeline import train_user_model, test_user_model

warnings.filterwarnings('ignore')

In [ ]:
SETTINGS_CACHE_DIR = os.environ.get('SETTINGS_CACHE_DIR', os.path.join('settings', 'cache'))
MODEL_CACHE_DIR = os.environ.get('MODEL_CACHE_DIR', 'saved_model')
LOG_SAVE_DIR = os.environ.get('LOG_SAVE_DIR', 'logs')
PRED_SAVE_DIR = os.environ.get('PRED_SAVE_DIR', 'predictions')

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # Add this line for CUDA
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
torch.multiprocessing.set_start_method('spawn')
device = f'cuda' if torch.cuda.is_available() else 'cpu'

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
torch.backends.cuda.enable_math_sdp(True)
# This key is an indicator of multiple things.
datetime_key = utils.get_datetime_key()

# Load the settings file, and save a backup in the cache directory.
with open(os.path.join('settings', f'local_test.json'), 'r') as fp:
    settings = json.load(fp)
utils.create_if_noexists(SETTINGS_CACHE_DIR)
with open(os.path.join(SETTINGS_CACHE_DIR, f'{datetime_key}.json'), 'w') as fp:
    json.dump(settings, fp)
    
print("device:", device)

device: cpu


In [ ]:
# Iterate through the multiple settings.
for setting_i, setting in enumerate(settings):
    SAVE_NAME = setting.get('save_name', None)
    
    train_traj_df = pd.read_hdf(setting['dataset']['train_traj_df'], key='trips')
    print("dataset:", setting['dataset']['train_traj_df'])

    scale = 4000
    # if "chengdu" in setting['dataset']['train_traj_df']:
    #     UTM_region = 48
    # if "xian" in setting['dataset']['train_traj_df']:
    #     UTM_region = 49
    if "geolife" in setting['dataset']['train_traj_df']:
        UTM_region = 50
            
    train_dataset = TrajFMDataset(traj_df=train_traj_df, UTM_region=UTM_region, scale = scale)
    
    poi_df = pd.read_hdf(setting['dataset']['poi_df'], key='pois')
    poi_embed = torch.from_numpy(np.load(setting['dataset']['poi_embed'])).float().to(device)
    
    poi_coors = poi_df[[X_COL, Y_COL]].to_numpy()
    poi_coors = (coord_transform_GPS_UTM(poi_coors, UTM_region) - train_dataset.spatial_middle_coord) / scale
    poi_coors = torch.tensor(poi_coors).float().to(device)

    # Build the learnable model.
    trajfm = TrajFM(poi_embed=poi_embed, 
                    poi_coors=poi_coors, 
                    UTM_region=UTM_region,
                    spatial_middle_coord = train_dataset.spatial_middle_coord, 
                    scale = scale, 
                    **setting['trajfm'],
                    user = setting["finetune"]["padder"]["params"]["num_users"]).to(device)

===SETTING 0/1===


In [ ]:
user_count = len(train_traj_df['user_id'].unique()))
traj_count = len(train_traj_df['traj_id'].unique())
traj_len = len(train_traj_df['seq_i'])
tao = train_traj_df['delta_t'].mean()

[  0   1   2   3   4   5   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  22  23  24  25  26  28  29  30  34  35  36  37  38  39  41  42
  43  44  46  50  51  52  56  62  65  67  68  71  73  74  78  81  82  83
  84  85  88  89  91  92  95  96  97 101 102 103 104 111 112 113 115 119
 125 126 128 134 140 141 142 144 147 153 154 155 163 167 168 169 174 179]


In [5]:
total_size = len(train_dataset)
train_size = int(0.6 * total_size)
val_test_size = total_size - train_size
val_size = int(0.5 * val_test_size)
test_size = val_test_size - val_size

train_dataset, val_test_dataset = random_split(train_dataset, [train_size, val_test_size])
val_dataset, test_dataset = random_split(val_test_dataset, [val_size, test_size])

In [ ]:
print(f"Loading model {SAVE_NAME}.pretrain")
downstreamtask = setting['finetune']['padder']['name']
trajfm.load_state_dict(torch.load(os.path.join(MODEL_CACHE_DIR, f'{SAVE_NAME}.{downstreamtask}'), map_location=device))

padder = fetch_task_padder(padder_name=setting['finetune']['padder']['name'], padder_params=setting['finetune']['padder']['params'])

train_dataloader = DataLoader(train_dataset, collate_fn=padder, **setting['finetune']['dataloader'])
val_dataloader = DataLoader(val_dataset, collate_fn=padder, **setting['finetune']['dataloader'])
test_dataloader = DataLoader(test_dataset, collate_fn=padder, **setting['finetune']['dataloader'])

In [ ]:
train_log, saved_model_state_dict = train_user_model(model=trajfm, 
                                                        train_dataloader=train_dataloader, 
                                                        val_dataloader=val_dataloader,
                                                        device = device, 
                                                        **setting['finetune']['config'])

if setting['finetune'].get('save', False):
    # save model
    utils.create_if_noexists(MODEL_CACHE_DIR)
    torch.save(saved_model_state_dict, os.path.join(MODEL_CACHE_DIR, f'{SAVE_NAME}.{downstreamtask}'))
    
    # save log
    log_dir = os.path.join(LOG_SAVE_DIR, SAVE_NAME)
    utils.create_if_noexists(log_dir)
    log_path = os.path.join(log_dir, f'{SAVE_NAME}_{downstreamtask}.csv')
    file_exists = os.path.exists(log_path)
    train_log.to_csv(log_path, mode='a', header=not file_exists, index=False)

In [ ]:
print(f"loading {SAVE_NAME}.{downstreamtask}")
trajfm.load_state_dict(torch.load(os.path.join(MODEL_CACHE_DIR, f'{SAVE_NAME}.{downstreamtask}'), map_location=device))

metrics, _ = test_user_model(model=trajfm, dataloader=test_dataloader, device = device)

for key, value in metrics.items():
    print(f"{key}: {round(value * 100, 3)}%,")

loading V_L300_U90.tul


Testing/Validating:   0%|          | 0/218 [00:00<?, ?it/s]

Testing/Validating: 100%|██████████| 218/218 [00:23<00:00,  9.20it/s]

ACC@1: 64.249%,
ACC@5: 87.643%,
Macro-R: 48.096%,
Macro-P: 47.513%,
Macro-F1: 47.009%,
